# 2. Two other common optimization algorithms

How does Bayesian optimization perform in comparison with other optimizing schemes? In this section, two relatively common algorithms from the [`scipy.optimize`](https://docs.scipy.org/doc/scipy/reference/optimize.html) package are used to tackle the same problem.

We start by importing the relevant packages:

In [1]:
import numpy as np
import scipy.optimize

np.random.seed(123)

Subsequently, the data input scaler is calibrated.

In [2]:
from sklearn.preprocessing import StandardScaler
scaler_bds = [[30., 50.], [34.18, 50.], [30., 200.], [34.18, 200.]]
scaler = StandardScaler()
scaler.fit(scaler_bds)

StandardScaler()

Then, the objective function is defined. Note that we have to include the back-transformation to the original design domain inside of the function, unlike when BO was used.

In [3]:
def gap_size(x):
    x = scaler.inverse_transform(x)
    open("a_Design_var1.txt", "w").write(str(x[0]))
    open("a_Design_var2.txt", "w").write(str(x[1]))
    # os.system("abaqus cae noGUI=2D-get.py")
    # The above line triggers the actual simulation. It is advised to uncomment this line only when the resources are available to perform the optimization procedure in full.
    return np.array(float(open("b_Objective_c_gap.txt", "r").read().strip()))

A callback function is defined in order to keep track of the iterations.

In [4]:
n_eval = 1
def callbackfun(x):
    global n_eval
    np.savetxt('ProgDir_NM/in_iter_%d.csv' % n_eval, scaler.inverse_transform(x), delimiter=",")
    # np.savetxt('ProgDir/out_iter_%d.csv' % n_eval, np.array([g(x)])) # In order to store the output per iteration, one more function evaluation is needed.
    n_eval += 1

### 2.1 Nelder-Mead simplex descent optimization

The Nelder-Mead optimization process starts with an initial value. Note that the maximum number of function evaluations should be equal to the same number in the BO case for comparison purposes.

In [5]:
x0 = scaler.transform(np.array([[32., 150.]]))

min_NM = scipy.optimize.minimize(gap_size, x0, method='Nelder-Mead', callback=callbackfun, options={'maxfev': 20, 'return_all': True})

The optimization object is stored after the optimization process.

In [6]:
open('ProgDir_NM/min_obj.txt', "w").write(str(min_NM))
np.savetxt('ProgDir_NM/in_history.csv', scaler.inverse_transform(min_NM['allvecs']), delimiter=',')

### 2.2 L-BFGS-B optimization

This optimization process is also a local optimization process with the additional benefit of being able to respect boundaries, unlike Nelder-Mead. We therefore supply the appropriate (scaled) bounds to perform this optimization. A similar callback function is constructed to keep the records, and the optimization object is saved accordingly after a similar number of iterations as with the previous applications.

In [7]:
n_eval = 1
def callbackfun_2(x):
    global n_eval
    np.savetxt('ProgDir_LBFGSB/in_iter_%d.csv' % n_eval, scaler.inverse_transform(x), delimiter=",")
    # np.savetxt('ProgDir/out_iter_%d.csv' % n_eval, np.array([g(x)])) # In order to store the output per iteration, one more function evaluation is needed.
    n_eval += 1

x0 = scaler.transform(np.array([[32., 150.]]))
bds = scipy.optimize.Bounds(scaler.transform(np.array([[30., 50.]])).flatten(), scaler.transform(np.array([[34.18, 200.]])).flatten())
min_LBFGSB = scipy.optimize.minimize(gap_size, x0, method='L-BFGS-B', callback=callbackfun_2, bounds=bds, options={'maxfun': 20})

open('ProgDir_LBFGSB/min_obj.txt', "w").write(str(min_LBFGSB))

282

In [8]:
print('All cells have been run.')

All cells have been run.
